# Transformando variáveis para árvores de decisão

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# abrindo arquivo
data = pd.read_csv('demo01.csv')
data

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,qt_pessoas_residencia,mau
0,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,58.832877,3.106849,1,0,0,0,2.0,False
1,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
2,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,1.0,False
3,M,Y,Y,0,Working,Higher education,Married,House / apartment,46.224658,2.106849,1,1,1,1,2.0,False
4,F,Y,N,0,Working,Incomplete higher,Married,House / apartment,29.230137,3.021918,1,0,0,0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16645,F,N,Y,0,Working,Secondary / secondary special,Civil marriage,House / apartment,54.109589,9.884932,1,0,0,0,2.0,True
16646,F,N,Y,0,Commercial associate,Secondary / secondary special,Married,House / apartment,43.389041,7.380822,1,1,1,0,2.0,True
16647,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,30.005479,9.800000,1,1,0,0,2.0,True
16648,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,30.005479,9.800000,1,1,0,0,2.0,True


In [ ]:
#verificando os tipos de variáveis / temos 8 tipos de variáveis para modificar e deixar em formato númerico.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16650 entries, 0 to 16649
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   16650 non-null  object 
 1   posse_de_veiculo       16650 non-null  object 
 2   posse_de_imovel        16650 non-null  object 
 3   qtd_filhos             16650 non-null  int64  
 4   tipo_renda             16650 non-null  object 
 5   educacao               16650 non-null  object 
 6   estado_civil           16650 non-null  object 
 7   tipo_residencia        16650 non-null  object 
 8   idade                  16650 non-null  float64
 9   tempo_emprego          16650 non-null  float64
 10  possui_celular         16650 non-null  int64  
 11  possui_fone_comercial  16650 non-null  int64  
 12  possui_fone            16650 non-null  int64  
 13  possui_email           16650 non-null  int64  
 14  qt_pessoas_residencia  16650 non-null  float64
 15  ma

In [ ]:
# verifcando quantidade de dados faltantes do df.
print('linhas e colunas: ', data.shape)
data.isna().sum()

linhas e colunas:  (16650, 16)


sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
possui_celular           0
possui_fone_comercial    0
possui_fone              0
possui_email             0
qt_pessoas_residencia    0
mau                      0
dtype: int64

In [ ]:
#verificando duplicatas  **obs: não irei tratar as duplicatas devido a o df não ter um id de cada cliente para diferenciar as linhas .
print(data.duplicated(subset=data.columns).value_counts())


True     9880
False    6770
dtype: int64


In [ ]:
# verificando a distribuição da variável "mau"
print(data.mau.value_counts())
percentual_mau = (data['mau'].value_counts()/len(data['mau']) *100)
percentual_mau

False    16260
True       390
Name: mau, dtype: int64


False    97.657658
True      2.342342
Name: mau, dtype: float64

#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.
3. Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}.

In [ ]:
#criando um pd.DataFrame() inserindo os tipos de variáveis das colunas neste df.
df = data
df_colu = pd.DataFrame(data.dtypes)
df_colu

,0
sexo,object
posse_de_veiculo,object
posse_de_imovel,object
qtd_filhos,int64
tipo_renda,object
educacao,object
estado_civil,object
tipo_residencia,object
idade,float64
tempo_emprego,float64


In [ ]:
#ultilizando o laço for para inserir uma nova coluna e adiconar a quantidade de variáveis nessta coluna
for i in data.columns:
  df_colu.loc[i, 'qtd_variaveis'] = data[i].nunique()

df_colu

,0,qtd_variaveis
sexo,object,2.0
posse_de_veiculo,object,2.0
posse_de_imovel,object,2.0
qtd_filhos,int64,8.0
tipo_renda,object,5.0
educacao,object,5.0
estado_civil,object,5.0
tipo_residencia,object,6.0
idade,float64,5298.0
tempo_emprego,float64,3005.0


In [ ]:
#criando variáveis dummies pra aquelas variáveis que são catégorias
df = pd.get_dummies(df, columns = ['tipo_renda','educacao','estado_civil','tipo_residencia'], drop_first = True)


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [ ]:
df.sexo = df.sexo.map({'M': 0, 'F': 1})
df.posse_de_veiculo = df.posse_de_veiculo.map({'Y':1, 'N': 0})
df.posse_de_imovel = df.posse_de_imovel.map({'Y': 1, 'N': 0})
df.mau = df.mau.astype(int)

In [ ]:
# verificando se todas as variáveis foram alteradas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16650 entries, 0 to 16649
Data columns (total 29 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   sexo                                    16650 non-null  int64  
 1   posse_de_veiculo                        16650 non-null  int64  
 2   posse_de_imovel                         16650 non-null  int64  
 3   qtd_filhos                              16650 non-null  int64  
 4   idade                                   16650 non-null  float64
 5   tempo_emprego                           16650 non-null  float64
 6   possui_celular                          16650 non-null  int64  
 7   possui_fone_comercial                   16650 non-null  int64  
 8   possui_fone                             16650 non-null  int64  
 9   possui_email                            16650 non-null  int64  
 10  qt_pessoas_residencia                   16650 non-null  fl

In [ ]:
# verificando data frame
df

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,idade,tempo_emprego,possui_celular,possui_fone_comercial,possui_fone,possui_email,...,educacao_Secondary / secondary special,estado_civil_Married,estado_civil_Separated,estado_civil_Single / not married,estado_civil_Widow,tipo_residencia_House / apartment,tipo_residencia_Municipal apartment,tipo_residencia_Office apartment,tipo_residencia_Rented apartment,tipo_residencia_With parents
0,0,1,1,0,58.832877,3.106849,1,0,0,0,...,1,1,0,0,0,1,0,0,0,0
1,1,0,1,0,52.356164,8.358904,1,0,1,1,...,1,0,0,1,0,1,0,0,0,0
2,1,0,1,0,52.356164,8.358904,1,0,1,1,...,1,0,0,1,0,1,0,0,0,0
3,0,1,1,0,46.224658,2.106849,1,1,1,1,...,0,1,0,0,0,1,0,0,0,0
4,1,1,0,0,29.230137,3.021918,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16645,1,0,1,0,54.109589,9.884932,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
16646,1,0,1,0,43.389041,7.380822,1,1,1,0,...,1,1,0,0,0,1,0,0,0,0
16647,0,1,1,0,30.005479,9.800000,1,1,0,0,...,1,1,0,0,0,1,0,0,0,0
16648,0,1,1,0,30.005479,9.800000,1,1,0,0,...,1,1,0,0,0,1,0,0,0,0


0    16260
1      390
Name: mau, dtype: int64

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [ ]:
# realizando verificação de correlação
tabela_email = pd.crosstab(df['possui_email'],df['mau'])
print(tabela_email)

correlacao_email = df['possui_email'].corr(df['mau'] == 1 )
print(correlacao_email)

mau               0    1
possui_email            
0             14810  360
1              1450   30
-0.00651185612529542


In [ ]:
# realizando verificação por valor na variável  possui_email
for i,x in enumerate(df.loc[df['mau'] == 1,'possui_email'].value_counts()):
    print(i, x, (x / df.loc[df['mau'] == 1,'possui_email'].value_counts().sum()))

0 360 0.9230769230769231
1 30 0.07692307692307693


In [ ]:
# realizando verificação por valor na variável posse_de_veículo

for i,x in enumerate(df.loc[df['mau'] == 1,'posse_de_veiculo'].value_counts()):
    print(i, x, (x / df.loc[df['mau'] == 1,'posse_de_veiculo'].value_counts().sum()))

0 247 0.6333333333333333
1 143 0.36666666666666664


In [ ]:
#realizando tabela para verificação de correlação
tabela_veiculo = pd.crosstab(df.posse_de_veiculo, df.mau)
print(tabela_veiculo)

correlacao_veiculo = df.posse_de_veiculo.corr(df.mau == 1)
correlacao_veiculo

mau                  0    1
posse_de_veiculo           
0                 9931  247
1                 6329  143


-0.007003056328371742

## As duas variáveis se mostraram com baixa correlação porém a que mais obteve valores de mau pagadores circustancialmente foi a variável posse de veículo

#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [ ]:
df.to_csv('demo01_tratado.csv')